Let's activate the virtual env

In [ ]:
!source ../.venv/bin/activate

Install package needed

In [ ]:
%pip install langgraph langchain langchain-aws dotenv

Import necessary modules

In [ ]:
import os
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, START, END
from langchain.tools import Tool
from langgraph.graph.message import add_messages
from typing import Annotated
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import ToolNode, tools_condition

Let's create a few tools

In [ ]:
from langchain_core.tools import tool

@tool
def covert_currency(value: float, currency:str, convert_to: str) -> float:
    "convert the value passed in a specific currency to the desired currency"
    if currency == "USD" and convert_to == "EUR":
        return value * 0.92
    else:
        return value*2
@tool        
def update_database():
    "update the database with the new information"
    return "database updated"

tools = [covert_currency,update_database]

Resgister the tools

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]  # Track conversation with LLM for analysis


graph_builder = StateGraph(AgentState)

llm = init_chat_model(
    "us.amazon.nova-lite-v1:0",
    model_provider="bedrock_converse",
)

llm_with_tools = llm.bind_tools(tools)


Let's define the LLM to use

In [ ]:


def chatbot(state: AgentState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

tool_node = ToolNode(tools)


creating the nodes and edges

In [ ]:
graph_builder.add_node("chatbot", chatbot)

graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

graph_builder.add_edge("tools", "chatbot")

graph_builder.add_edge(START, "chatbot")

#graph_builder.add_edge("chatbot", END)

Compile the agent

In [ ]:
graph = graph_builder.compile()

visualizing the graph

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

Run the agent

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(".env")

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

exit
